# Data Cleaning and Preparation

## 7.1 Handling Missing Data

对于数值型数据,pandas使用浮点值NaN(Not a Number来表示缺失值)

In [2]:
import numpy as np
import pandas as pd
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

Python内建的None值在对象数组中也被当作NA处理

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

|函数名|描述|
|------|----|
|dropna|根据每个标签的值是否是缺失数据来筛选轴标签，并根据允许丢失的数据量来确定阈值|
|fillna|用某些值填充的数据或使用插值方法(如'ffill'或'bfill')|
|isnull|返回表明哪些值是缺失值的布尔值|
|notnull|isnull的反函数|

### 7.1.1 Filtering Out Missing Data

In [8]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna() # 过滤后的索引不变
# 等价于 data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

当处理DataFrame对象时，dropna默认会删除包含缺失值的行

In [13]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna() # 原来的变量不会改变
cleaned

,0,1,2
0,1.0,6.5,3.0


In [14]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
data.dropna(how='all') # 将删除所有值均为NA的行

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [16]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
data.dropna(axis=1, how='all') # 删除所有值均为NA的列

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


若要保留包含一定数量的观察值的行，可以使用thresh参数来表示

In [19]:
df= pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1]=NA
df.iloc[:2,2]=NA
df

,0,1,2
0,1.281854,NaN,NaN
1,-0.877157,NaN,NaN
2,0.815553,NaN,-1.364519
3,1.230652,NaN,-0.750578
4,-1.744318,0.658630,-1.519121
5,0.600262,2.949627,-0.690294
6,0.751587,-0.219150,-0.648063


In [23]:
df.dropna() # 过滤掉缺失值的行

,0,1,2
4,-1.744318,0.658630,-1.519121
5,0.600262,2.949627,-0.690294
6,0.751587,-0.219150,-0.648063


In [24]:
df.dropna(thresh=2) # 保留两行观察值

,0,1,2
2,0.815553,NaN,-1.364519
3,1.230652,NaN,-0.750578
4,-1.744318,0.658630,-1.519121
5,0.600262,2.949627,-0.690294
6,0.751587,-0.219150,-0.648063


### 7.1.2 Filling In Missing Data

主要使用fillna方法来补全缺失值，调用fillna时，可以使用一个常数来代替缺失值

In [26]:
df.fillna(0) # 将缺失值用0来替代

,0,1,2
0,1.281854,0.000000,0.000000
1,-0.877157,0.000000,0.000000
2,0.815553,0.000000,-1.364519
3,1.230652,0.000000,-0.750578
4,-1.744318,0.658630,-1.519121
5,0.600262,2.949627,-0.690294
6,0.751587,-0.219150,-0.648063


In [29]:
# 也可以用字典为不同的列设定不同的填充值
df.fillna({1: 0.5, 2: 0}) # 第一列设定为0.5填充，第二列设定为0填充

,0,1,2
0,1.281854,0.500000,0.000000
1,-0.877157,0.500000,0.000000
2,0.815553,0.500000,-1.364519
3,1.230652,0.500000,-0.750578
4,-1.744318,0.658630,-1.519121
5,0.600262,2.949627,-0.690294
6,0.751587,-0.219150,-0.648063


fillna返回的是一个新的对象，但可以传入inplace=True修改已经存在的对象

In [32]:
_ = df.fillna(0, inplace=True) # 改变df本身
df

,0,1,2
0,1.281854,0.000000,0.000000
1,-0.877157,0.000000,0.000000
2,0.815553,0.000000,-1.364519
3,1.230652,0.000000,-0.750578
4,-1.744318,0.658630,-1.519121
5,0.600262,2.949627,-0.690294
6,0.751587,-0.219150,-0.648063


In [37]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA # 将索引列1号的第2行以后用NA填充
df.iloc[4:, 2] = NA # 将索引列2号的第4行以后用NA填充
df

,0,1,2
0,0.525490,0.460673,1.337807
1,1.533623,0.189514,0.104073
2,-1.005903,NaN,-0.548376
3,-0.121539,NaN,-0.601334
4,-0.929284,NaN,NaN
5,0.999322,NaN,NaN


In [38]:
df.fillna(method='ffill') # 填充方法默认是前向填充，即向下填充

,0,1,2
0,0.525490,0.460673,1.337807
1,1.533623,0.189514,0.104073
2,-1.005903,0.189514,-0.548376
3,-0.121539,0.189514,-0.601334
4,-0.929284,0.189514,-0.601334
5,0.999322,0.189514,-0.601334


In [39]:
df.fillna(method='ffill', limit=2) # 向下填充的最大范围设定为2

,0,1,2
0,0.525490,0.460673,1.337807
1,1.533623,0.189514,0.104073
2,-1.005903,0.189514,-0.548376
3,-0.121539,0.189514,-0.601334
4,-0.929284,NaN,-0.601334
5,0.999322,NaN,-0.601334


In [40]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean()) # 用3个数的平均数填充缺失值 (1+3.5+7)/3=3.833333

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

|参数|描述|
|----|----|
|value|标量值或字典型对象用于填充缺失值|
|method|插值方法，如果没有其他参数，默认是'ffill'|
|axis|需要填充的轴，默认为axis=0|
|inplace|修改被调用的对象，而不是生成一个备份|
|limit|用于向前或向后填充时最大的填充范围|

## 7.2 Data Transformation

### 7.2.1 Removing Duplicates

In [42]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data # 索引5，6重复

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的duplicated方法返回的是一个布尔值Series，这个Series反映的是每一行是否存在重复（与之前出现过的行相同）

In [44]:
data.duplicated() # 返回布尔值

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicates返回的是DataFrame，内容是duplicated返回数组中为False的部分

In [47]:
data.drop_duplicates() # 返回不重复的部分

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [51]:
# 添加一个额外的列
data['v1'] = range(7)
data.drop_duplicates(['k1']) # 删除k1里的重复值

,k1,k2,v1
0,one,1,0
1,two,1,1


In [53]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


duplicated和drop_duplicates默认都是保留第一个观测到的值，传入参数keep='last'将会返回最后一个

In [56]:
data.drop_duplicates(['k1','k2'])

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


In [54]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 Transforming Data Using a Function or Mapping

In [57]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [58]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map方法接收一个函数或一个包含映射关系的字典型对象

In [59]:
lowercased = data['food'].str.lower() # Series的str.lower()方法将每个值都转换为小写
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [61]:
data['animal'] = lowercased.map(meat_to_animal) # 按照字典的指定进行匹配
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [63]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
# x.lower()变小写
# x 为变量(来自food)，food的迭代

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [64]:
meat_to_animal['bacon']

'pig'

### 7.2.3 Replacing Values

使用fillna填充缺失值是通用值替换的特殊案例；map可以用来修改一个对象中的子集的值

In [67]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [68]:
data.replace(-999, np.nan) # 将-999替换为NaN

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [69]:
data.replace([-999, -1000], np.nan) # 一次代替多个值，可传入一个列表和替代值

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [70]:
# 要将不同的值替换为不同的值，可以传入替代值的列表
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [72]:
# 参数也可以通过字典传递
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

data.repplace方法与data.str.replace方法是不同的，data.str.replace是对字符串进行按元素替代的

### 7.2.4 Renaming Axis Indexes

In [74]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [76]:
# 先定义一个匿名函数，取出[0:4)，字符串的前4位，并全大写
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [78]:
data.index = data.index.map(transform) # 赋值给index，修改DataFrame
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


若创建数据集转换后的版本，并且不修改原有的数据集，即方法rename

In [81]:
data.rename(index=str.title, columns=str.upper)
# str.title 首字母全大写
# str.upper 全字母大写

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [83]:
data # 原数据集不变

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [86]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'}) # rename结合字典型对象，位轴标签的子集提供新的值

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [90]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True) #  inplace=True 修改i原有的数据集
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 Discretization and Binning

In [91]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [93]:
bins = [18, 25, 35, 60, 100] # 临界值(18,25),(25,35),(35,60),(60,100)
cats = pd.cut(ages, bins) # 进行分组，返回的对象是一个特殊的对象
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [95]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [97]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [99]:
pd.value_counts(cats) # 统计每个里面的个数

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

pd.value_counts(cats)是对pandas.cut的结果中的箱数量的计数

In [102]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) # 右边不封闭

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [104]:
pd.cut(ages, [18, 26, 36, 61, 100], right=True) # 右边封闭

[(18, 26], (18, 26], (18, 26], (26, 36], (18, 26], ..., (26, 36], (36, 61], (36, 61], (36, 61], (26, 36]]
Length: 12
Categories (4, interval[int64]): [(18, 26] < (26, 36] < (36, 61] < (61, 100]]

可向labels选项传递一个列表或数组来传入自定义的箱名

In [106]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names) # 定义箱名

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [109]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2) # 将data分成4份

[(0.49, 0.72], (0.72, 0.95], (0.034, 0.26], (0.034, 0.26], (0.49, 0.72], ..., (0.034, 0.26], (0.26, 0.49], (0.26, 0.49], (0.034, 0.26], (0.034, 0.26]]
Length: 20
Categories (4, interval[float64]): [(0.034, 0.26] < (0.26, 0.49] < (0.49, 0.72] < (0.72, 0.95]]

precision=2的选项将十进制精度限制在两位

qcut是一个与分箱密切相关的函数，它基于样本分位数进行分箱。取决于数据的分布，使用cut通常不会使每个箱具有相同数据量的数据点，qcut使用样本的分位数，获得等长的箱

In [114]:
data = np.random.randn(1000)  # 正态分布
cats = pd.qcut(data, 4)  # 切成四份
cats

[(-2.69, -0.614], (-2.69, -0.614], (0.668, 4.007], (0.668, 4.007], (0.668, 4.007], ..., (-2.69, -0.614], (-2.69, -0.614], (-2.69, -0.614], (-0.614, 0.00783], (-2.69, -0.614]]
Length: 1000
Categories (4, interval[float64]): [(-2.69, -0.614] < (-0.614, 0.00783] < (0.00783, 0.668] < (0.668, 4.007]]

In [115]:
pd.value_counts(cats)

(0.668, 4.007]       250
(0.00783, 0.668]     250
(-0.614, 0.00783]    250
(-2.69, -0.614]      250
dtype: int64

In [116]:
# 与cut类似，可传入自定义的分位数(0和1之间的数据，包括边)
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.217, 0.00783], (-1.217, 0.00783], (1.308, 4.007], (0.00783, 1.308], (0.00783, 1.308], ..., (-2.69, -1.217], (-2.69, -1.217], (-2.69, -1.217], (-1.217, 0.00783], (-2.69, -1.217]]
Length: 1000
Categories (4, interval[float64]): [(-2.69, -1.217] < (-1.217, 0.00783] < (0.00783, 1.308] < (1.308, 4.007]]

### 7.2.6 Detecting and Filtering Outliers

In [118]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()
# describe()函数就是返回这两个核心数据结构的统计变量

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.002784,-0.009893,-0.012407,0.013654
std,1.003532,1.030677,0.984919,0.992065
min,-3.447209,-3.452307,-3.001733,-2.757781
25%,-0.707263,-0.720082,-0.707855,-0.637295
50%,-0.007926,-0.058648,-0.021517,0.032919
75%,0.693082,0.681546,0.689808,0.662905
max,3.348227,3.475457,3.305793,2.767207


In [119]:
# 找出一列中绝对值大于3的值
col = data[2]
col[np.abs(col) > 3]

76    -3.001733
358    3.305793
Name: 2, dtype: float64

选出所有值大于3或小于-3的行，可对布尔值DataFrame使用any方法

In [121]:
data[(np.abs(data) > 3).any(1)] # 某一行有一个大于3，该行将被留下

,0,1,2,3
19,0.830761,3.122250,-1.242498,-0.555545
67,0.983587,3.074070,-0.890098,-0.385787
76,3.348227,-0.762294,-3.001733,0.234119
119,0.531241,3.048410,-0.963787,-1.325491
183,-0.378724,3.045034,0.798215,0.761239
358,-1.077309,-0.058544,3.305793,-0.335391
378,3.181243,0.257630,-1.030933,-0.188796
592,-3.447209,-0.340586,0.095308,0.741946
607,0.942623,-3.452307,0.196425,0.256816
902,-0.072689,3.475457,-0.617080,-0.068703


In [122]:
data[np.abs(data) > 3] = np.sign(data) * 3
# np.sign(data) 若为正 ---> +3
# np.sign(data) 若为负 ---> -3

In [123]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.002867,-0.010206,-0.012711,0.013654
std,1.000428,1.026898,0.983930,0.992065
min,-3.000000,-3.000000,-3.000000,-2.757781
25%,-0.707263,-0.720082,-0.707855,-0.637295
50%,-0.007926,-0.058648,-0.021517,0.032919
75%,0.693082,0.681546,0.689808,0.662905
max,3.000000,3.000000,3.000000,2.767207


np.sign(data)根据数据中的值的正负分别生成1和-1的数值

In [125]:
np.sign(data).head()  # 输出前五行

,0,1,2,3
0,1.0,1.0,1.0,1.0
1,1.0,-1.0,1.0,1.0
2,-1.0,1.0,-1.0,1.0
3,-1.0,-1.0,1.0,1.0
4,1.0,1.0,-1.0,1.0


### 7.2.7 Permutation and Random Sampling

numpy.random.permutation对DataFrame中的Series或行进行置换（随机重排序）；在调用permutation时根据想要的轴长度可以产生一个表示新顺序的整数数组

In [127]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5) # 轴长度，生成一个长度为5的随机数组
sampler

array([3, 1, 0, 2, 4])

In [128]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


整数数组可以用在基于iloc的索引或等价的take函数中

In [130]:
df.take(sampler) # 按索引进行排序 [3, 1, 0, 2, 4]

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19


In [134]:
df.sample(n=3) # 生成一个3行的随机样本，不可重复
# sample(序列a，n)
# 功能：从序列a中随机抽取n个元素，并将n个元素生以list形式返回。

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
3,12,13,14,15


要生成一个带有替代值的样本(允许有重复选择)，将replace=True传入sample方法

In [136]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True) # 随机抽样，可重复
draws

3    6
0    5
4    4
4    4
2   -1
4    4
2   -1
3    6
3    6
0    5
dtype: int64

### 7.2.8 Computing Indicator/Dummy Variables

如果DataFrame中的一列有k个不同的值，则可在衍生一个k列的值为1和0的矩阵或DataFrame,pandas中的get_dummies函数可实现此功能

In [137]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key']) # 将abc变成三列，用1标记

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [139]:
dummies = pd.get_dummies(df['key'], prefix='key') #  prefix='key' 列上面加上前缀key
dummies 

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [141]:
df_with_dummy = df[['data1']].join(dummies) # 与其他类合并
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [143]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10] # [0:10)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [145]:
all_genres = []
for x in movies.genres:  # 列表加入列表
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)  # 取唯一值

In [147]:
genres # 将所有标签单独地取出来

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [149]:
zero_matrix = np.zeros((len(movies), len(genres)))
# 生成的行数等于movies，列数等于电影类别的全为0的数组
dummies = pd.DataFrame(zero_matrix, columns=genres)
# 电影类别作为列，数组的作为值

在这张表中，我们统计，每一行对应的类别，如果属于该类别，就标记为1，如果不属于，就标记为0

In [150]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [178]:
for i, gen in enumerate(movies.genres): # i 表示行序号，gen 表示值
    # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [179]:
movies_windic = movies.join(dummies.add_prefix('Genre_')) # 把标记好的列加入原来的列中
movies_windic.iloc[0] # 第0行

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

将get_dummies与cut等离散化函数结合使用是统计应用的一个有用的办法

In [180]:
np.random.seed(12345)
# 使用numpy.random.seed来设置随机种子以确保示例的确定性
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [181]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 String Manipulation

### 7.3.1 String Object Methods

使用split方法拆分成多块

In [182]:
val = 'a,b,  guido' 
val.split(',')  # 用逗号分隔

['a', 'b', '  guido']

split和strip一起使用，用于清除空格（包括换行）

In [183]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [184]:
first, second, third = pieces
first + '::' + second + '::' + third # 字符串可通过加法和分隔符连接在一起，加号合并字符串
# 等价于 '::'.join(pieces)

'a::b::guido'

使用Python中的in关键字是检测子字符串的最佳办法，index，find也可

In [185]:
'guido' in val

True

In [186]:
val.index(',')

1

In [187]:
val.find(':')

-1

In [188]:
val.index(':')

ValueError: substring not found

#### 注意：find和index的区别在于index在字符串没有找到时会抛出一个异常(而find返回的时-1)

In [189]:
val.count(',') # 返回某个特定的子字符串在字符串中出现的次数

2

replace将用一种模式替代另一种模式。它通常也用于传入空字符串来删除某个模式

In [190]:
val.replace(',', '::') # 用'::' 代替 ','

'a::b::  guido'

In [191]:
val.replace(',', '') # 用 '' 代替 ','

'ab  guido'

### 7.3 Regular Expressions

描述一个或多个空白字符的正则表达式是\s+

In [193]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

当调用re.split('\s+',text),正则表达式首先会被编译，然后正则表达式的split方法在传入文本上的被调用；可使用re.compile自行编译，形成一个可复用的正则表达式对象

In [194]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

使用findall方法获得的是一个所有匹配正则表达式的模式的列表

In [195]:
regex.findall(text)

['    ', '\t ', '  \t']

#### 为了在正则表达式中避免转义字符\的影响，可以使用原生字符串语法，比如r'C:\x'或者用等价的'C:\\x'

|方法|描述|
|----|----|
|findall|返回的是字符串中所有的匹配项|
|search|返回的仅仅是第一个匹配项|
|match|它只在字符串的起始位置进行匹配|

In [205]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [208]:
regex.findall(text) # 在文本上使用会生成一个电子邮件地址的列表

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [209]:
m = regex.search(text) # 返回的是文本中第一个匹配到的电子邮件地址
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [207]:
text[m.start():m.end()]

'dave@google.com'

regex.match只在模式出现于字符串起始位置时进行匹配，如果没有匹配到，返回None

In [210]:
print(regex.match(text)) 

None


sub会返回一个新的字符串，原字符串中的模式会被以个新的字符串替代

In [211]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [212]:
# 用括号将正则表达式括起来，返回的就是分开的元组
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [213]:
m = regex.match('wesm@bright.net')
m.groups() # 返回的是模式组件的元组

('wesm', 'bright', 'net')

In [214]:
# 当模式可以分组的时候，findall返回的是包含元组的列表
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub也可以使用特殊符号，如\1和\2，访问每个匹配对象中的分组.符号\1代表的是第一个匹配分组，\2代表的是第二个匹配分组，以此类推

In [215]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### 7.3.3 Vectorized String Functions in pandas

In [216]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [217]:
data.isnull() # 返回表明哪些值是缺失值的布尔值

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

使用data.map将字符串和有效的正则表达式方法(以lambda或其他函数的方式传递)应用到每个值上，但在NA(null)值上会失败

In [219]:
data.str.contains('gmail')
# Series的str属性，可以跳过NA值判断字符串中是否有gmail

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

正则表达式也可以结合任意的re模块选项使用，例如IGNORECASE

In [220]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [221]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [222]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [227]:
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [230]:
data.str.get(1)
# matches是bool型的Series对象，每一个索引只对应一个值，所以在get(1)时就不存在值对应，返回nan

Dave       a
Steve      t
Rob        o
Wes      NaN
dtype: object

In [232]:
matches.str[0]
# False,True不支持索引(即bool值)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [233]:
data.str[:5]  # 使用字符串切片的类似语法进行向量化切片

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

## Conclusion